In [3]:
#import packages and files
import pandas as pd
import plotly
import plotly.graph_objects as go
import keyring
import psycopg2
import pandas.io.sql as sqlio
from ipywidgets import HBox, VBox, Label, Text, Layout, Combobox
import yaml
from yaml import SafeDumper
import math
import re
import os
import collections
import Wales_LAEP_Lib as laep

ModuleNotFoundError: No module named 'psycopg2'

In [ ]:
# Connect to database if needed
db = laep.WalesLeapDB() 
db.disconnect()
db = laep.WalesLeapDB() 
password = db.get_password()
db.connect(host='10.126.48.132', port='5432', user='waleslaep@psgaz-d-wlp-uks', dbname='wales_laep', password=password)

In [ ]:
#choose which table/view to read in from the database
sql = "select * from pembrokeshire_calliope.model_run_sankey_inputs_latest"

In [ ]:
#choose which scenario and iteration to visualise
scenario = '2050_high_demand' #input("Enter scenario then press enter: ") 
iteration = 'Run_6_7_' #input("Enter iteration number then press enter: ")

In [ ]:
#read in data
#df = pd.read_csv('2019_baseline_sankey_25032022.csv') #use this for non database csvs - ensure csv is in sankey inputs format
df = sqlio.read_sql_query(sql, db.conn) #use this for straight from database

df = df[(df['scenario'] == scenario) & (df['iteration'] == iteration)]
df.head()

,input,weight,output,scenario,iteration
1,Anaerobic digestion,0.086,Primary electricity,2050_high_demand,Run_6_7_
2,Biomass,0.018,Biomass boiler to electricity,2050_high_demand,Run_6_7_
3,Biomass boiler to electricity,0.005,Electricity,2050_high_demand,Run_6_7_
4,Biomass,1.478,Biomass boiler to heat,2050_high_demand,Run_6_7_
5,Biomass boiler to heat,1.034,Heat,2050_high_demand,Run_6_7_


In [ ]:
display(df)

,input,weight,output,scenario,iteration
1,Anaerobic digestion,0.086,Primary electricity,2050_high_demand,Run_6_7_
2,Biomass,0.018,Biomass boiler to electricity,2050_high_demand,Run_6_7_
3,Biomass boiler to electricity,0.005,Electricity,2050_high_demand,Run_6_7_
4,Biomass,1.478,Biomass boiler to heat,2050_high_demand,Run_6_7_
5,Biomass boiler to heat,1.034,Heat,2050_high_demand,Run_6_7_
6,Biomass import,1.495,Biomass,2050_high_demand,Run_6_7_
7,Electricity,566.867,Electricity demand,2050_high_demand,Run_6_7_
8,Heat,1362.064,Heat demand,2050_high_demand,Run_6_7_
9,Hydrogen,41.000,Industrial hydrogen demand,2050_high_demand,Run_6_7_
10,Hydrogen,41.000,Industrial hydrogen demand,2050_high_demand,Run_6_7_


In [ ]:
#filter out unwanted values here
#function to remove unwanted rows/unnecessary detail - could be improved/automated 

# #removing unwanted rows
df = df[df['input'].str.contains('elec_lines') == False]
df = df[df['input'].str.contains('Hydrogen refueller') == False]
df = df[df['input'].str.contains('substation') == False]
df = df[df['input'].str.contains('mileage') == False]
df = df[df['output'].str.contains('mileage') == False]
df = df[df['input'].str.contains('transport') == False]
df = df[df['input'].str.contains('Biomass import') == False]
df = df[df['input'].str.contains('export') == False]
df = df[df['output'].str.contains('elec_lines') == False]
df = df[df['output'].str.contains('substation') == False]

# #editing rows
df.loc[(df['input']=='Primary electricity'), 'input'] = 'Electricity'
df.loc[(df['output']=='Hydrogen refueller'), 'output'] = 'Hydrogen refueller to transport'
df.loc[(df['output']=='Primary electricity'), 'output'] = 'Electricity'
df.loc[(df['input']=='Grid electricity'), 'input'] = 'Electricity'
df.loc[(df['output']=='Grid electricity'), 'output'] = 'Electricity'
df.loc[(df['output']=='Grid electricity'), 'output'] = 'Electricity'

#remove very small flows
# df = df[df['weight'] < 1) == False]

########################################################
# general editing syntax:
#to change the numeric value of the line with "X" as the input:
#df.loc[(df['input']=='X'), 'weight'] = 'new_value'
#to delete the row with "Y" as the output
#df = df[df['output'].str.contains('Y') == False]
#to remove a specific row with index number Z:
# df = df.drop(Z)

#check results are looking ok
display(df)

,input,weight,output,scenario,iteration
1,Anaerobic digestion,0.086,Electricity,2050_high_demand,Run_6_7_
2,Biomass,0.018,Biomass boiler to electricity,2050_high_demand,Run_6_7_
3,Biomass boiler to electricity,0.005,Electricity,2050_high_demand,Run_6_7_
4,Biomass,1.478,Biomass boiler to heat,2050_high_demand,Run_6_7_
5,Biomass boiler to heat,1.034,Heat,2050_high_demand,Run_6_7_
7,Electricity,566.867,Electricity demand,2050_high_demand,Run_6_7_
8,Heat,1362.064,Heat demand,2050_high_demand,Run_6_7_
9,Hydrogen,41.000,Industrial hydrogen demand,2050_high_demand,Run_6_7_
10,Hydrogen,41.000,Industrial hydrogen demand,2050_high_demand,Run_6_7_
16,Electricity,255.379,Electrolyser,2050_high_demand,Run_6_7_


In [ ]:
#make any further corrections to the data here, repeating process of above cell

In [ ]:
#check the data looks ok. if not, repeat above cells
display(df)

,input,weight,output,scenario,iteration
1,Anaerobic digestion,0.086,Electricity,2050_high_demand,Run_6_7_
2,Biomass,0.018,Biomass boiler to electricity,2050_high_demand,Run_6_7_
3,Biomass boiler to electricity,0.005,Electricity,2050_high_demand,Run_6_7_
4,Biomass,1.478,Biomass boiler to heat,2050_high_demand,Run_6_7_
5,Biomass boiler to heat,1.034,Heat,2050_high_demand,Run_6_7_
7,Electricity,566.867,Electricity demand,2050_high_demand,Run_6_7_
8,Heat,1362.064,Heat demand,2050_high_demand,Run_6_7_
9,Hydrogen,41.000,Industrial hydrogen demand,2050_high_demand,Run_6_7_
10,Hydrogen,41.000,Industrial hydrogen demand,2050_high_demand,Run_6_7_
16,Electricity,255.379,Electrolyser,2050_high_demand,Run_6_7_


In [ ]:
# #for generating random colours for the nodes
# import random
# label_colours = []
# hexs = random.sample(range(0, 16777215), len(my_sankey['source']))
# for j in range(len(my_sankey['source'])):
#     no1 = random.randint(0,256)
#     no2 = random.randint(0,256)
#     no3 = random.randint(0,256)
#     label_colours.append("rgb("+str(no1)+", "+str(no2)+", "+str(no3)+")")    

In [ ]:
#for a specific set of colours, define a colour dictionary
#add any technologies in the format:
#{'Technology name [as in the dataframe]':'hex colour code'}

colour_dict = {
    'Anaerobic digestion':'#808080',
    'Battery':'#7f0000',
    'Biomass':'#006400',
    'Hydrogen':'#808000',
    'Boiler':'#808000',
    'Electricity':'#483d8b',
    'Biomass boiler to electricity':'#3cb371',
    'Biomass boiler to heat':'#008080',
    'Heat':'#000080',
    'Heat demand':'#4682b4',
    'Electricity transport':'#cd853f',
    'Petrol/diesel':'#cd853f',
    'Electricity demand':'#7e7a36',
    'Hydrogen heat demand':'#32cd32',
    'Industrial hydrogen demand':'#8b008b',
    'Electrolyser':'#b03060',
    'EV chargers':'#d2b48c',
    'Bioenergy and waste':'#d2b48c',
    'Ground PV':'#ff4500',
    'Heat network':'#00ff00',
    'Heat pump':'#00ced1',
    'Hydroelectricity':'#ff8c00',
    'Hydrogen CCGT':'#ffd700',
    'Oil/solid/coal':'#ffd700',
    'Hydrogen OCGT':'#0000ff',
    'Hydrogen boiler to heat':'#a020f0',
    'Transport demand':'#a020f0',
    'Hydrogen CHP':'#f08080',
    'Gas boiler':'#f08080',
    'Hydrogen export':'#adff2f',
    'Hydrogen import':'#dc143c',
    'Natural gas':'#dc143c',
    'Hydrogen refueller to transport':'#b0c4de',
    'Road vehicles':'#b0c4de',
    'Hydrogen storage tank':'#f0e68c',
    'Hydrogen transport':'#1e90ff',
    'National grid export':'#ff00ff',
    'National grid import':'#d6f0ef',
    'Onshore wind':'#98fb98',
    'Resistance heating':'#7b68ee',
    'Rail':'#7b68ee',
    'Rooftop PV':'#ee82ee',
    'Sewage gas':'#98fb98',
    'Solar thermal':'#7fffd4',
    'Hydrogen heat':'#f0e68c',
    'Landfill gas':'#ffc0cb'
}

In [ ]:
#function for assigining colours to nodes as defined by their label
def assign_colour(label_input):
    for label, colour in colour_dict.items():
        if label == label_input:
            return colour

In [ ]:
#set up inputs for sankey builder
#source of original code (edited here): https://medium.com/codex/how-to-automatically-generate-data-structure-for-sankey-diagrams-6082e332139f
def get_sankey(data,path,value_col):
    sankey_data = {
    'label':[],
    'source': [],
    'target' : [],
    'value' : []

    }
    counter = 0
    while (counter < len(path) - 1):
        for parent in data[path[counter]].unique():
            sankey_data['label'].append(parent)
            for inputs in data[data[path[counter]] == parent][path[counter]].unique():
                for outputs in data[data[path[counter]] == parent][path[counter+1]]:
                    weights = data[(data[path[counter+1]] == outputs) & (data[path[counter]] == inputs)][value_col].sum()
                    sankey_data['label'].append(outputs)
                    sankey_data['source'].append(sankey_data['label'].index(parent))
                    sankey_data['target'].append(sankey_data['label'].index(outputs))
                    sankey_data['value'].append(round(float(weights)))
        counter +=1
    return sankey_data
my_sankey = get_sankey(df, ['input','output'],'weight')

#add weight value to labels and assign colours
label_vals = []
colours_list = []
for i in my_sankey['label']:
    colours_list.append(assign_colour(i))
    if i in df['input'].values:
        temp = df[df['input']==i]['weight'].sum()
        label_vals.append(i+ ": "+str(round(float(temp))))
    elif i in df['output'].values:
        temp = df[df['output']==i]['weight'].sum()
        label_vals.append(i+ ": "+str(round(float(temp))))

In [ ]:
#build sankey
#source of original code (edited here): https://medium.com/codex/how-to-automatically-generate-data-structure-for-sankey-diagrams-6082e332139f
fig = go.Figure(data=[go.Sankey(
    arrangement="perpendicular", #other layout options available
    node = dict(
      pad = 30, #spacing between nodes
      thickness = 15,#thickness of nodes
      line = dict(color = "black", width = 0.5), #node outline
      label = label_vals,
      color = colours_list #change this to a given colour for them to be monochromatic or to label_colours for random colours
    ),
    link = dict(
      source = my_sankey['source'],
      target = my_sankey['target'],
      value = my_sankey['value']
  ))])
fig.update_layout(height=1150, width=1500, margin={'t':150,'b':20,'l':10,'r':150}, font= dict(size=25))